In [1]:
"""
STILL WORK IN PROGRESS!!!!!!
"""

import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import os
import datetime
import glob


In [12]:
def read_file(file):
    """
    read file line by line
    """
    lines = [line.rstrip() for line in open(file, 'r', encoding='ISO-8859-15')]
    return lines

def find_idx(lines, substring):
    """
    find the line indexes containing a substring
    """
    idx = []
    count = 0
    for line in lines:
        if substring in line:
            idx.append(count)
        count += 1
    return idx

def keep_cols_by_name(df, substring):
    df = df.filter(regex=substring)
    return df


def unstack_double_header(df, col_index):
    """
    transform df with double headers into single header
    """

    # Unstack two headers
    df.iloc[col_index] = df.iloc[col_index].str.strip()
    cols = df.iloc[col_index][2::].unique() # skip first two columns 
    cols = (list(set([col.strip() for col in cols])))

    result = pd.DataFrame()
    for col in cols:

        # get the indexes of the desired columns
        idx = np.where(df.iloc[0] == col)[0]
        # insert first column (key)
        idx = np.insert(idx, 0, [0, 1], axis=0)

        # filter just the desired columns and removes lines from headers
        df_aux = df.loc[2::,idx]
        df_aux.columns = df.loc[1, idx]

        df_aux["new_col"] = col
        result = pd.concat([result, df_aux])

    return result

def stack_same_tables_in_same_file(idxs, lines):
    result = pd.DataFrame()
    for i in range(1, len(idxs)):
        fst_idx=idxs[i-1]+2
        last_idx=idxs[i]-1

        df = pd.DataFrame([line.split(";") for line in lines[fst_idx:last_idx]])
        df.iloc[1, 1:] = df.iloc[1, :-1].values # correct second line header, witch is offset by 1

        aux = unstack_double_header(df, 0)
        aux["table"] = lines[fst_idx-2]
        result = pd.concat([result, aux])

        print(f"\t Time: {datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')} - Iteraction: {i} - First idx = {fst_idx} - Last idx = {last_idx} - Shape before unstack: {df.shape} - Shape after unstack: {aux.shape} - Shape after append: {result.shape} ")
    return result

def df_janitor(df):

    # Correct new columns added names
    #df = df.rename({"new_col":"Faixa Etaria", "table":"Sexo"}, axis='columns')

    # adjust content of the lines of the column Faixa Etaria
    #df["Faixa Etaria"] = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', s)) for s in df["Faixa Etaria"]]

    # adjust content of the lines of the column Sexo
    #df["Sexo"] = [re.sub("Sexo Trabalhador = ", '', s) for s in df["Sexo"]]
    #df["Sexo"] = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', s)) for s in df["Sexo"]]

    # remove unwanted lines
    to_remove = ['{ñ class}', 'Total', '', 'Seleções vigentes', 'Variável', 'Ano', 'Vínculo Ativo 31/12']
    df = df[~df.iloc[:,0].isin(to_remove)]
    #df = df[~df["Faixa Etaria"].isin(to_remove)]
    #df = df[~df["Sexo"].isin(to_remove)]

    # transform string nan to effective nan
    df = df.replace("NaN", np.nan)

    # adjust columns names
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]

    # adjust line names
    df.iloc[:,0] = df.iloc[:,0].str.split(':').str[1]

    # remove special chars
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]
    df.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df.columns]

    return df

def load_idadeXsexoXsetor(file):
    lines = read_file(file)
    substring = "Sexo Trabalhador = "
    idxs = find_idx(lines, substring)
    # adding the last line of the file (+1 to be ok with the logic below)
    idxs.append(len(lines)+1)

    # read the file and stack the tables
    result = stack_same_tables_in_same_file(idxs, lines)

    # clean the output
    df = df_janitor(result)

    df["arquivo"] = os.path.split(file)[1]

    return df

In [17]:
path = os.getcwd()
extension = 'csv'
os.chdir(path)
files_csv = glob.glob('*.{}'.format(extension))
output_file="load_sexoXescolaridadeXcboXsmX.csv"

df_final = pd.DataFrame()
for i in files_csv:
    if i != output_file:
        aux = load_idadeXsexoXsetor(i)
        df_final = pd.concat([df_final, aux])
        print(f"Time: {datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')} - File {i}")

df_final.to_csv(output_file)

                                                  0    \
0                                    CBO 2002 Família   
1                                                       
2     1112:DIRIGENTES GERAIS DA ADMINISTRACAO PUBLICA   
3                                                       
4                                                       
...                                               ...   
7917                                                    
7918                                                    
7919                                                    
7920                                                    
7921                                                    

                          1                       2                   3    \
0      Faixa Remun Média (SM)           01:Analfabeto       01:Analfabeto   
1     110020:Porto Velho - RO  120040:Rio Branco - AC  130260:Manaus - AM   
2                       Total                       0                   0   
3      